In [2]:
import pandas as pd
from collections import Counter
import os
import ast
import requests
from bs4 import BeautifulSoup
import time


## Parsing new data for old tags

In [3]:
tags_to_keep = ['туризм', 'саяхат', 'қазақстан', 'тоқаев қасым-жомарт', 'видеолар',
       'коронавирус', 'ресей', 'соңғы жаңалықтар', 'құқық қорғау органдары',
       'сот', 'карантин', 'мектеп', 'нұр-сұлтан', 'алматы', 'ақш', 'вакцина',
       'пандемия', 'денсаулық', 'бокс', 'астана', 'тағайындау', 'министр',
       'спорт', 'шымкент ', 'жекпе-жек', 'балалар', 'ақорда', 'көліктер',
       'полиция', 'қасым-жомарт тоқаев', 'қылмыс', 'алматы облысы', 'әкім',
       'ұрлық', 'пневмония']

In [4]:
def get_articles_list(tag):
  default_url = 'https://kaz.tengrinews.kz/tag/'
  cnt = 0
  page_num = 1
  all_links = []

  while cnt < 300:
    if page_num == 1:
      tag_url = default_url + tag
    else:
      tag_url = default_url + tag + '/page' + str(page_num)

    try:
      r = requests.get(tag_url, timeout=(3, 5))
      if r.status_code == 404:
        print(f"Page not found: tag - {tag}, page num - {page_num}, number of articles - {cnt}")
        break
      r.raise_for_status()
    except requests.exceptions.RequestException as e:
      print(f"Error no page {tag_url}: {e}")
      break

    soup = BeautifulSoup(r.text, 'html.parser')
    spans = soup.find_all('span', class_='content_main_item_title')
    if not spans:
      print(f"Нет статей на странице: {tag_url}")
      break

    for span in spans:
      link = span.find('a')
      if link and link.get('href'):
        all_links.append('https://kaz.tengrinews.kz' + link.get('href'))
        cnt += 1

    page_num += 1

  return all_links


def get_text_tags(url):
  try:
    r = requests.get(url, timeout=(3,5))
    r.raise_for_status()
  except requests.exceptions.Timeout:
    print(f"Timeout при загрузке: {url}")
    return "", []
  except requests.exceptions.RequestException as e:
    print(f"Error downloading {url}: {e}")
    return "", []
  soup = BeautifulSoup(r.text, 'html.parser')
  text_div = soup.find('div', class_='content_main_text')
  elements = text_div.find_all(['p', 'li'])
  article_text = []
  for el in elements:
    for tag in el.find_all(['a', 'img', 'em', 'strong', 'span']):
        tag.decompose()
    text = el.get_text(strip=True)
    if text:
        article_text.append(text)
  tags = []
  tags_spans = soup.find('div', class_='content_main_text_tags')
  if tags_spans:
      tags_spans = tags_spans.find_all('a')
      for tag in tags_spans:
          tags.append(tag.get_text())

  return '\n'.join(article_text), tags


In [ ]:
all_articles_df = pd.DataFrame()

for tag in tags_to_keep:

  print(tag)

  article_of_each_tag = {
    'text': [],
    'tags': [],
    'tags_list': [],
    'url': []
  }

  all_links = get_articles_list(tag)
  time.sleep(1)

  for link in all_links:
    text, tags = get_text_tags(link)
    time.sleep(1)
    tags_str = f"[{', '.join(tags)}]"
    article_of_each_tag['text'].append(text)
    article_of_each_tag['tags'].append(tags_str)
    article_of_each_tag['tags_list'].append(tags)
    article_of_each_tag['url'].append(link)

  tag_df = pd.DataFrame(article_of_each_tag)
  all_articles_df = pd.concat([all_articles_df, tag_df], ignore_index=True)

  print(f'{tag} ✅')


туризм
туризм ✅


In [6]:
new_all_articles = pd.DataFrame(all_articles_df)

In [ ]:
new_all_articles.to_csv(os.path.join('dataset', 'new_all_articles.csv'), index = False)

## Prep new data for old tags

In [265]:
import ast

new_articles = pd.read_csv('dataset\\new_all_articles.csv')
new_articles['tags_list'] = new_articles['tags_list'].apply(ast.literal_eval)

In [266]:
new_articles.head()

,text,tags,tags_list,url
0,Туризм және спорт вице-министрі Ержан Еркінбае...,"[Туризм, Астана]","[Туризм, Астана]",https://kaz.tengrinews.kz/my-country/astanada-...
1,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,"[Туризм, оқыс оқиға]","[Туризм, оқыс оқиға]",https://kaz.tengrinews.kz/my-country/fukuok-as...
2,Шетелдік азаматтардың Қазақстандағы мәртебесін...,"[Көші-қон, Туризм]","[Көші-қон, Туризм]",https://kaz.tengrinews.kz/my-country/sheteldk-...
3,Бурабай ұлттық паркі Instagram-парақшасында өз...,"[табиғат, Туризм]","[табиғат, Туризм]",https://kaz.tengrinews.kz/my-country/burabay-a...
4,"Президент Тоқаевтың сөзінше, туризм экономикан...","[Қасым-Жомарт Тоқаев, ақорда, Туризм]","[Қасым-Жомарт Тоқаев, ақорда, Туризм]",https://kaz.tengrinews.kz/my-country/kalyip-ba...


In [267]:
new_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8700 entries, 0 to 8699
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       8698 non-null   object
 1   tags       8700 non-null   object
 2   tags_list  8700 non-null   object
 3   url        8700 non-null   object
dtypes: object(4)
memory usage: 272.0+ KB


In [268]:
from collections import Counter

all_rows = []
for row in new_articles['tags_list']:
  for word in row:
    all_rows.extend(row)

counter = Counter(all_rows)
new_articles_topics = pd.Series(counter)
new_articles_topics.describe()

count     359.000000
mean      168.913649
std       334.798982
min        30.000000
25%        45.000000
50%        60.000000
75%       120.000000
max      3015.000000
dtype: float64

In [269]:
new_articles_topics.sort_values(ascending=False)[:35]

коронавирус                     3015
Алматы                          2428
Қазақстан                       2205
Астана                          1650
спорт                           1335
Денсаулық                       1200
Туризм                          1181
пандемия                        1155
қылмыс                          1140
Полиция                         1078
Қасым-Жомарт Тоқаев             1065
Бокс                            1050
вакцина                          990
АҚШ                              975
карантин                         971
ақорда                           960
пневмония                        930
Сот                              900
Балалар                          885
Ресей                            870
әкім                             870
көліктер                         855
Нұр-Сұлтан                       825
жекпе-жек                        825
Саяхат                           810
ұрлық                            795
Тағайындау                       765
м

In [270]:
new_articles_topics_keep = new_articles_topics[new_articles_topics >= 400].index
new_articles_topics_keep

Index(['Туризм', 'Астана', 'Қасым-Жомарт Тоқаев', 'ақорда', 'Қазақстан',
       'Саяхат', 'спорт', 'Алматы', 'Қытай', 'Полиция', 'кеңес', 'көліктер',
       'видеолар', 'коронавирус', 'вакцина', 'Денсаулық сақтау министрлігі',
       'Денсаулық', 'пневмония', 'Ресей', 'АҚШ', 'қылмыс', 'Сот', 'Тағайындау',
       'карантин', 'Мектеп', 'Нұр-Сұлтан', 'ұрлық', 'министр', 'пандемия',
       'Балалар', 'Бокс', 'әкім', 'жекпе-жек'],
      dtype='object')

In [271]:
def drop_tags(row):
  new_list = []
  for topic in row:
    if topic in new_articles_topics_keep:
      new_list.append(topic)
  return new_list

new_articles['tags_list_cut'] = new_articles['tags_list'].apply(drop_tags)
new_articles.head(10)

,text,tags,tags_list,url,tags_list_cut
0,Туризм және спорт вице-министрі Ержан Еркінбае...,"[Туризм, Астана]","[Туризм, Астана]",https://kaz.tengrinews.kz/my-country/astanada-...,"[Туризм, Астана]"
1,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,"[Туризм, оқыс оқиға]","[Туризм, оқыс оқиға]",https://kaz.tengrinews.kz/my-country/fukuok-as...,[Туризм]
2,Шетелдік азаматтардың Қазақстандағы мәртебесін...,"[Көші-қон, Туризм]","[Көші-қон, Туризм]",https://kaz.tengrinews.kz/my-country/sheteldk-...,[Туризм]
3,Бурабай ұлттық паркі Instagram-парақшасында өз...,"[табиғат, Туризм]","[табиғат, Туризм]",https://kaz.tengrinews.kz/my-country/burabay-a...,[Туризм]
4,"Президент Тоқаевтың сөзінше, туризм экономикан...","[Қасым-Жомарт Тоқаев, ақорда, Туризм]","[Қасым-Жомарт Тоқаев, ақорда, Туризм]",https://kaz.tengrinews.kz/my-country/kalyip-ba...,"[Қасым-Жомарт Тоқаев, ақорда, Туризм]"
5,Қазақстанда үш жаңа әуежайдың құрылысы басталд...,"[әуежай, Қазақстан, Туризм]","[әуежай, Қазақстан, Туризм]",https://kaz.tengrinews.kz/my-country/kazakstan...,"[Қазақстан, Туризм]"
6,Қазақстандық әлеуметтік желілерді таңғалдырды:...,"[Қазақстан, күй, Туризм]","[Қазақстан, күй, Туризм]",https://kaz.tengrinews.kz/my-country/kazakstan...,"[Қазақстан, Туризм]"
7,Въетнамда демалғысы келген астаналықтар алаяққ...,"[Алаяқтық, Туризм]","[Алаяқтық, Туризм]",https://kaz.tengrinews.kz/my-country/shetelde-...,[Туризм]
8,Елдің туризм саласындағы биылғы әлеуеті жөнінд...,"[Туризм, Қазақстан]","[Туризм, Қазақстан]",https://kaz.tengrinews.kz/my-country/biyil-kaz...,"[Туризм, Қазақстан]"
9,Қазақстанда ішкі туристер саны артты. Бұл тура...,"[Туризм, Қазақстан]","[Туризм, Қазақстан]",https://kaz.tengrinews.kz/my-country/kansha-ka...,"[Туризм, Қазақстан]"


In [272]:
new_articles.dropna(subset=['tags_list_cut'], inplace=True)

In [273]:
def lower_all_words(list_of_words):
    return [word.lower() for word in list_of_words]
    
new_articles['tags_list_cut'] = new_articles['tags_list_cut'].apply(lower_all_words)

In [274]:
new_articles.head()

,text,tags,tags_list,url,tags_list_cut
0,Туризм және спорт вице-министрі Ержан Еркінбае...,"[Туризм, Астана]","[Туризм, Астана]",https://kaz.tengrinews.kz/my-country/astanada-...,"[туризм, астана]"
1,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,"[Туризм, оқыс оқиға]","[Туризм, оқыс оқиға]",https://kaz.tengrinews.kz/my-country/fukuok-as...,[туризм]
2,Шетелдік азаматтардың Қазақстандағы мәртебесін...,"[Көші-қон, Туризм]","[Көші-қон, Туризм]",https://kaz.tengrinews.kz/my-country/sheteldk-...,[туризм]
3,Бурабай ұлттық паркі Instagram-парақшасында өз...,"[табиғат, Туризм]","[табиғат, Туризм]",https://kaz.tengrinews.kz/my-country/burabay-a...,[туризм]
4,"Президент Тоқаевтың сөзінше, туризм экономикан...","[Қасым-Жомарт Тоқаев, ақорда, Туризм]","[Қасым-Жомарт Тоқаев, ақорда, Туризм]",https://kaz.tengrinews.kz/my-country/kalyip-ba...,"[қасым-жомарт тоқаев, ақорда, туризм]"


## Kaggle dataset

In [321]:
kaggle_df = pd.read_csv('dataset\\tengri_news.csv')
kaggle_df.head()

,title,url,tags,text
0,Әлемдегі ең танымал туристік бағыттар аталды,https://kaz.tengritravel.kz/around-the-world/a...,"['туризм', 'рейтинг', 'саяхат']",Tripadvisor нұсқасы бойынша Индонезияның Бали ...
1,2020 жылдың қорытындысы бойынша Қазақстан халқ...,https://kaz.tengrinews.kz/kazakhstan_news/2020...,"['қазақстан', 'халық саны']",2021 жылы елдегі халық саны 18 миллион 877 мың...
2,Қазақстандықтар 2021 жылды қарсы алып жатыр,https://kaz.tengrinews.kz/kazakhstan_news/kaza...,"['мереке', 'жаңа жыл']",Қазақстандықтар 2021 жылды қарсы алып жатыр. 2...
3,2022 жылы Балқаш көлінде демалу қанша тұрады,https://kaz.tengritravel.kz/my-country/2022-jy...,"['туризм', 'балқаш', 'турист', 'саяхат']",2022 жылдың жазғы маусымында Балқаш көлінде де...
4,Тоқаев қазақстандықтарды 2022 жылы не күтіп тұ...,https://kaz.tengrinews.kz/kazakhstan_news/toka...,"['тоқаев қасым-жомарт', 'жаңа жыл']",Президент Қасым-Жомарт Тоқаев қазақстандықтард...


In [322]:
kaggle_df["tags_list"] = kaggle_df["tags"].apply(ast.literal_eval)
kaggle_df.head()

,title,url,tags,text,tags_list
0,Әлемдегі ең танымал туристік бағыттар аталды,https://kaz.tengritravel.kz/around-the-world/a...,"['туризм', 'рейтинг', 'саяхат']",Tripadvisor нұсқасы бойынша Индонезияның Бали ...,"[туризм, рейтинг, саяхат]"
1,2020 жылдың қорытындысы бойынша Қазақстан халқ...,https://kaz.tengrinews.kz/kazakhstan_news/2020...,"['қазақстан', 'халық саны']",2021 жылы елдегі халық саны 18 миллион 877 мың...,"[қазақстан, халық саны]"
2,Қазақстандықтар 2021 жылды қарсы алып жатыр,https://kaz.tengrinews.kz/kazakhstan_news/kaza...,"['мереке', 'жаңа жыл']",Қазақстандықтар 2021 жылды қарсы алып жатыр. 2...,"[мереке, жаңа жыл]"
3,2022 жылы Балқаш көлінде демалу қанша тұрады,https://kaz.tengritravel.kz/my-country/2022-jy...,"['туризм', 'балқаш', 'турист', 'саяхат']",2022 жылдың жазғы маусымында Балқаш көлінде де...,"[туризм, балқаш, турист, саяхат]"
4,Тоқаев қазақстандықтарды 2022 жылы не күтіп тұ...,https://kaz.tengrinews.kz/kazakhstan_news/toka...,"['тоқаев қасым-жомарт', 'жаңа жыл']",Президент Қасым-Жомарт Тоқаев қазақстандықтард...,"[тоқаев қасым-жомарт, жаңа жыл]"


In [323]:
all_rows = []
for row in kaggle_df['tags_list']:
  all_rows.extend(row)

counter = Counter(all_rows)


In [324]:
kaggle_df_topics = pd.Series(counter)
kaggle_df_topics.describe()

count    2460.000000
mean       34.698780
std       126.335369
min         1.000000
25%         2.000000
50%         6.000000
75%        23.000000
max      3231.000000
dtype: float64

In [325]:
kaggle_df_topics.sort_values(ascending=False).head(35)

коронавирус               3231
алматы                    2369
соңғы жаңалықтар          1774
қазақстан                 1717
қылмыс                    1697
карантин                  1238
вакцина                   1059
ресей                      849
нұр-сұлтан                 660
жекпе-жек                  648
полиция                    639
қасым-жомарт тоқаев        639
ақш                        615
тоқаев қасым-жомарт        602
астана                     593
балалар                    556
әкім                       535
пандемия                   520
ақорда                     504
ұрлық                      500
пневмония                  482
құқық қорғау органдары     474
спорт                      466
бокс                       448
туризм                     445
министр                    444
алматы облысы              441
мектеп                     439
тағайындау                 429
видеолар                   422
шымкент                    420
денсаулық                  417
көліктер

In [326]:
kaggle_df_topics_keep = kaggle_df_topics[kaggle_df_topics >= 400].index
kaggle_df_topics_keep

Index(['туризм', 'саяхат', 'қазақстан', 'тоқаев қасым-жомарт', 'видеолар',
       'коронавирус', 'ресей', 'соңғы жаңалықтар', 'құқық қорғау органдары',
       'сот', 'карантин', 'мектеп', 'нұр-сұлтан', 'алматы', 'ақш', 'вакцина',
       'пандемия', 'денсаулық', 'бокс', 'астана', 'тағайындау', 'министр',
       'спорт', 'шымкент ', 'жекпе-жек', 'балалар', 'ақорда', 'көліктер',
       'полиция', 'қасым-жомарт тоқаев', 'қылмыс', 'алматы облысы', 'әкім',
       'ұрлық', 'пневмония'],
      dtype='object')

In [327]:
len(kaggle_df_topics_keep)

35

In [328]:
kaggle_df_balanced = pd.DataFrame()
for topic in kaggle_df_topics_keep:
  filtered_df = kaggle_df[kaggle_df['tags'].str.contains(topic, case=False, na=False)]
  topic_sample = filtered_df.sample(n=400, random_state=42)
  kaggle_df_balanced = pd.concat([kaggle_df_balanced, topic_sample])
kaggle_df_balanced.head()

,title,url,tags,text,tags_list
12331,Дубайда әлемдегі ең биік панорамалық бассейн а...,https://kaz.tengritravel.kz/around-the-world/d...,"['туризм', 'бассейн', 'дубай', 'саяхат']",Дубайда 360 градусқа қарайтын әлемдегі ең биік...,"[туризм, бассейн, дубай, саяхат]"
17068,Туроператорлар Мысырдың қазақстандықтар үшін қ...,https://kaz.tengritravel.kz/my-country/turoper...,"['шекара', 'мысыр', 'рейс', 'туризм', 'коронав...",Қазақстандық туристер өткен жылдың қарашасынан...,"[шекара, мысыр, рейс, туризм, коронавирус]"
5426,Нью-Йоркте үрейлі аттракцион пайда болады,https://kaz.tengritravel.kz/around-the-world/n...,"['туризм', 'нью-йорк', 'турист', 'ақш']",Нью-Йорк тұрғындары мен қонақтары енді қаланы ...,"[туризм, нью-йорк, турист, ақш]"
23718,БАӘ-де жылдың осы уақыты үшін қалыпсыз аптап ы...,https://kaz.tengritravel.kz/around-the-world/b...,"['туризм', 'ыстық', 'біріккен араб әмірліктері...",Біріккен Араб Әмірліктеріндегі ауа райы темпер...,"[туризм, ыстық, біріккен араб әмірліктері, ауа..."
3775,Ердоған Түркия туризмінің ең үлкен мәселесін а...,https://kaz.tengritravel.kz/travel-notes/erdog...,"['табыс', 'түркия', 'инфляция', 'туризм', 'реж...","Туризм ""мақсаттарды барынша жүзеге асыруға"" бе...","[табыс, түркия, инфляция, туризм, режеп тайып ..."


In [329]:
def drop_kaggle_tags(row):
  new_list = []
  for topic in row:
    if topic in kaggle_df_topics_keep:
      new_list.append(topic)
  return new_list

In [330]:
kaggle_df_balanced['tags_list_cut'] = kaggle_df_balanced['tags_list'].apply(drop_kaggle_tags)
kaggle_df_balanced.head()

,title,url,tags,text,tags_list,tags_list_cut
12331,Дубайда әлемдегі ең биік панорамалық бассейн а...,https://kaz.tengritravel.kz/around-the-world/d...,"['туризм', 'бассейн', 'дубай', 'саяхат']",Дубайда 360 градусқа қарайтын әлемдегі ең биік...,"[туризм, бассейн, дубай, саяхат]","[туризм, саяхат]"
17068,Туроператорлар Мысырдың қазақстандықтар үшін қ...,https://kaz.tengritravel.kz/my-country/turoper...,"['шекара', 'мысыр', 'рейс', 'туризм', 'коронав...",Қазақстандық туристер өткен жылдың қарашасынан...,"[шекара, мысыр, рейс, туризм, коронавирус]","[туризм, коронавирус]"
5426,Нью-Йоркте үрейлі аттракцион пайда болады,https://kaz.tengritravel.kz/around-the-world/n...,"['туризм', 'нью-йорк', 'турист', 'ақш']",Нью-Йорк тұрғындары мен қонақтары енді қаланы ...,"[туризм, нью-йорк, турист, ақш]","[туризм, ақш]"
23718,БАӘ-де жылдың осы уақыты үшін қалыпсыз аптап ы...,https://kaz.tengritravel.kz/around-the-world/b...,"['туризм', 'ыстық', 'біріккен араб әмірліктері...",Біріккен Араб Әмірліктеріндегі ауа райы темпер...,"[туризм, ыстық, біріккен араб әмірліктері, ауа...",[туризм]
3775,Ердоған Түркия туризмінің ең үлкен мәселесін а...,https://kaz.tengritravel.kz/travel-notes/erdog...,"['табыс', 'түркия', 'инфляция', 'туризм', 'реж...","Туризм ""мақсаттарды барынша жүзеге асыруға"" бе...","[табыс, түркия, инфляция, туризм, режеп тайып ...",[туризм]


In [331]:
kaggle_df_balanced.shape

(14000, 6)

In [332]:
kaggle_df_balanced = kaggle_df_balanced[kaggle_df_balanced['tags_list_cut'].apply(lambda x:len(x)>0)]

In [333]:
kaggle_df_balanced.shape

(13545, 6)

In [334]:
all_rows = []
for row in kaggle_df_balanced['tags_list_cut']:
  all_rows.extend(row)

counter = Counter(all_rows)
kaggle_topics = pd.Series(counter)
display(kaggle_topics.sort_values(ascending=False))

коронавирус               2325
соңғы жаңалықтар          1261
алматы                    1238
қылмыс                    1080
карантин                  1053
вакцина                    894
қазақстан                  870
жекпе-жек                  832
ұрлық                      738
нұр-сұлтан                 726
құқық қорғау органдары     717
бокс                       702
полиция                    672
ресей                      660
алматы облысы              660
балалар                    654
пандемия                   644
туризм                     643
ақорда                     637
тоқаев қасым-жомарт        622
астана                     592
видеолар                   581
саяхат                     580
пневмония                  575
мектеп                     570
тағайындау                 565
шымкент                    563
қасым-жомарт тоқаев        547
спорт                      535
ақш                        501
әкім                       492
денсаулық                  434
көліктер

In [335]:
kaggle_topics.describe()

count      35.000000
mean      720.600000
std       355.239869
min       308.000000
25%       564.000000
50%       643.000000
75%       732.000000
max      2325.000000
dtype: float64

## Concatenate dfs

In [346]:
text_tags_kaggle = kaggle_df_balanced[['text', 'tags_list_cut']]
text_tags_kaggle.reset_index(inplace=True, drop=True)

In [347]:
text_tags_kaggle.head()

,text,tags_list_cut
0,Дубайда 360 градусқа қарайтын әлемдегі ең биік...,"[туризм, саяхат]"
1,Қазақстандық туристер өткен жылдың қарашасынан...,"[туризм, коронавирус]"
2,Нью-Йорк тұрғындары мен қонақтары енді қаланы ...,"[туризм, ақш]"
3,Біріккен Араб Әмірліктеріндегі ауа райы темпер...,[туризм]
4,"Туризм ""мақсаттарды барынша жүзеге асыруға"" бе...",[туризм]


In [348]:
text_tags_new_articles = new_articles[['text', 'tags_list_cut']]
text_tags_new_articles.reset_index(drop=True, inplace=True)

In [349]:
text_tags_new_articles.head()

,text,tags_list_cut
0,Туризм және спорт вице-министрі Ержан Еркінбае...,"[туризм, астана]"
1,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,[туризм]
2,Шетелдік азаматтардың Қазақстандағы мәртебесін...,[туризм]
3,Бурабай ұлттық паркі Instagram-парақшасында өз...,[туризм]
4,"Президент Тоқаевтың сөзінше, туризм экономикан...","[қасым-жомарт тоқаев, ақорда, туризм]"


In [350]:
old_all_articles = pd.concat([text_tags_new_articles, text_tags_kaggle], ignore_index=True)
old_all_articles

,text,tags_list_cut
0,Туризм және спорт вице-министрі Ержан Еркінбае...,"[туризм, астана]"
1,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,[туризм]
2,Шетелдік азаматтардың Қазақстандағы мәртебесін...,[туризм]
3,Бурабай ұлттық паркі Instagram-парақшасында өз...,[туризм]
4,"Президент Тоқаевтың сөзінше, туризм экономикан...","[қасым-жомарт тоқаев, ақорда, туризм]"
...,...,...
22240,2020 жылғы 26 желтоқсанда тіркелген коронавиру...,"[пневмония, коронавирус]"
22241,Пандемия басталғалы 2 088 қазақстандық коронав...,"[пневмония, коронавирус, пандемия]"
22242,Қазақстанда бір тәулік ішінде коронавирус пен ...,"[пневмония, қазақстан, коронавирус]"
22243,Қазақстанда коронавирус пен пневмониядан тағы ...,"[пневмония, коронавирус]"


In [358]:
old_all_articles.drop_duplicates(subset='text', keep='last')

,text,tags_list_cut
281,Желіде Фукуок – Астана бағытындағы ұшақ құспен...,[туризм]
282,Шетелдік азаматтардың Қазақстандағы мәртебесін...,[туризм]
283,Бурабай ұлттық паркі Instagram-парақшасында өз...,[туризм]
284,"Президент Тоқаевтың сөзінше, туризм экономикан...","[қасым-жомарт тоқаев, ақорда, туризм]"
285,Қазақстанда үш жаңа әуежайдың құрылысы басталд...,"[қазақстан, туризм]"
...,...,...
22240,2020 жылғы 26 желтоқсанда тіркелген коронавиру...,"[пневмония, коронавирус]"
22241,Пандемия басталғалы 2 088 қазақстандық коронав...,"[пневмония, коронавирус, пандемия]"
22242,Қазақстанда бір тәулік ішінде коронавирус пен ...,"[пневмония, қазақстан, коронавирус]"
22243,Қазақстанда коронавирус пен пневмониядан тағы ...,"[пневмония, коронавирус]"


In [359]:
all_rows = []
for row in old_all_articles['tags_list_cut']:
  for word in row:
    all_rows.extend(row)

counter = Counter(all_rows)
old_all_articles_topics = pd.Series(counter)
old_all_articles_topics.describe()

count      38.000000
mean     2146.000000
std      1381.279802
min       300.000000
25%      1393.000000
50%      1915.000000
75%      2369.000000
max      8486.000000
dtype: float64

In [361]:
old_all_articles_topics.sort_values(ascending=False)

коронавирус                     8486
алматы                          4657
қазақстан                       3644
карантин                        3403
вакцина                         3089
қылмыс                          3008
соңғы жаңалықтар                2937
пандемия                        2674
жекпе-жек                       2456
ұрлық                           2373
пневмония                       2357
бокс                            2357
астана                          2350
нұр-сұлтан                      2287
полиция                         2271
туризм                          2241
ақорда                          2053
спорт                           2033
ресей                           1918
денсаулық                       1912
балалар                         1885
қасым-жомарт тоқаев             1847
құқық қорғау органдары          1722
тағайындау                      1632
ақш                             1596
видеолар                        1591
саяхат                          1531
м